# Search and download data from Digtal Africa

https://docs.digitalearthafrica.org/en/latest/data_specs/Landsat_WOfS_specs.html#

In [22]:
import sys, os, importlib
import rasterio, boto3

import pandas as pd
import geopandas as gpd

from botocore import UNSIGNED
from botocore.config import Config
from GOSTRocks.misc import tPrint

bucket = 'deafrica-services'
prefix = 'wofs_ls_summary_annual'
region = 'af-south-1'
s3client = boto3.client('s3', region_name='af-south-1', config=Config(signature_version=UNSIGNED))

In [23]:
# Define your input variables
iso3 = 'MWI'
year = 2020
product = '_frequency.tif' #ount_clear.tif, _count_wet.tif
out_folder = f"/home/wb411133/temp/{iso3}"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [24]:
#Read in extents of output tiles
in_extents = gpd.read_file("wofs_ls_summary_alltime-regions-deafrica-data.geojson")
in_extents['COL'] = in_extents['region_code'].apply(lambda x: int(x.split("_")[0]) + 181) # These additions transform numbers for downloads
in_extents['ROW'] = in_extents['region_code'].apply(lambda x: int(x.split("_")[1]) + 77)
in_extents['COL_ROW'] = in_extents.apply(lambda x: f"{x['COL']}_{x['ROW']}", axis=1)

#Read in country extent and select intersecting tiles
adm0 = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
sel_extent = adm0.loc[adm0['iso_a3'] == iso3]
sel_tiles = in_extents.loc[in_extents.intersects(sel_extent.unary_union)]

In [25]:
sel_tiles.head()

,region_code,label,count,geometry,COL,ROW,COL_ROW
2757,32_-13,32_-13,1,"POLYGON ((31.83872 -9.06690, 32.83368 -9.06690...",213,64,213_64
2762,32_-18,32_-18,1,"POLYGON ((31.83872 -12.89847, 32.83368 -12.898...",213,59,213_59
2763,32_-19,32_-19,1,"POLYGON ((31.83872 -13.67113, 32.83368 -13.671...",213,58,213_58
2839,33_-13,33_-13,1,"POLYGON ((32.83368 -9.06690, 33.82864 -9.06690...",214,64,214_64
2840,33_-14,33_-14,1,"POLYGON ((32.83368 -9.82949, 33.82864 -9.82949...",214,63,214_63


In [26]:
# Loop through the S3 bucket and get all the keys for files that are .tif 
more_results = True
try:
    del(token)
except:
    pass
loops = 0
good_res = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=token)
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects['IsTruncated']
    if more_results:
        token = objects['NextContinuationToken']
    loops += 1
    for res in objects['Contents']:
        if res['Key'].endswith('.tif'):
            good_res.append(res)

Completed loop: 0
Completed loop: 1
Completed loop: 2
Completed loop: 3
Completed loop: 4
Completed loop: 5
Completed loop: 6
Completed loop: 7
Completed loop: 8
Completed loop: 9
Completed loop: 10
Completed loop: 11
Completed loop: 12
Completed loop: 13
Completed loop: 14
Completed loop: 15
Completed loop: 16
Completed loop: 17
Completed loop: 18
Completed loop: 19
Completed loop: 20
Completed loop: 21
Completed loop: 22
Completed loop: 23
Completed loop: 24
Completed loop: 25
Completed loop: 26
Completed loop: 27
Completed loop: 28
Completed loop: 29
Completed loop: 30
Completed loop: 31
Completed loop: 32
Completed loop: 33
Completed loop: 34
Completed loop: 35
Completed loop: 36
Completed loop: 37
Completed loop: 38
Completed loop: 39
Completed loop: 40
Completed loop: 41
Completed loop: 42
Completed loop: 43
Completed loop: 44
Completed loop: 45
Completed loop: 46
Completed loop: 47
Completed loop: 48
Completed loop: 49
Completed loop: 50
Completed loop: 51
Completed loop: 52
Com

Completed loop: 416
Completed loop: 417
Completed loop: 418
Completed loop: 419
Completed loop: 420
Completed loop: 421
Completed loop: 422
Completed loop: 423
Completed loop: 424
Completed loop: 425
Completed loop: 426
Completed loop: 427
Completed loop: 428
Completed loop: 429
Completed loop: 430
Completed loop: 431
Completed loop: 432
Completed loop: 433
Completed loop: 434
Completed loop: 435
Completed loop: 436
Completed loop: 437
Completed loop: 438
Completed loop: 439
Completed loop: 440
Completed loop: 441
Completed loop: 442
Completed loop: 443
Completed loop: 444
Completed loop: 445
Completed loop: 446
Completed loop: 447
Completed loop: 448
Completed loop: 449
Completed loop: 450
Completed loop: 451
Completed loop: 452
Completed loop: 453
Completed loop: 454
Completed loop: 455
Completed loop: 456
Completed loop: 457
Completed loop: 458
Completed loop: 459
Completed loop: 460
Completed loop: 461
Completed loop: 462
Completed loop: 463
Completed loop: 464
Completed loop: 465


In [27]:
# Convert the results to a data frame
s3res = pd.DataFrame(good_res)
s3res['COL']  = s3res['Key'].apply(lambda x: int(x.split("/")[2][1:]))
s3res['ROW']  = s3res['Key'].apply(lambda x: int(x.split("/")[3][1:]))
s3res['COL_ROW'] = s3res.apply(lambda x: f"{x['COL']}_{x['ROW']}", axis=1)
s3res['TYPE'] = s3res['Key'].apply(lambda x: x.split("/")[-1][-14:])
s3res['YEAR'] = s3res['Key'].apply(lambda x: x.split("/")[4][:4])
s3res['Size'] = s3res['Size'].apply(lambda x: x/1024/1024)

s3res.head()

,Key,LastModified,ETag,Size,StorageClass,COL,ROW,COL_ROW,TYPE,YEAR
0,wofs_ls_summary_annual/1-0-0/x154/y096/2013--P...,2021-09-08 02:33:16+00:00,"""d32ed33cb46effee73892e43e5da2e98""",0.371935,STANDARD,154,96,154_96,ount_clear.tif,2013
1,wofs_ls_summary_annual/1-0-0/x154/y096/2013--P...,2021-09-08 02:33:16+00:00,"""e5eeea7c323a984c4bb159b1cb8ea3c1""",0.298257,STANDARD,154,96,154_96,_count_wet.tif,2013
2,wofs_ls_summary_annual/1-0-0/x154/y096/2013--P...,2021-09-08 02:33:18+00:00,"""a9ba372ec73d855e2783f285a0935834""",0.937539,STANDARD,154,96,154_96,_frequency.tif,2013
3,wofs_ls_summary_annual/1-0-0/x154/y096/2014--P...,2021-09-08 02:35:41+00:00,"""c5feac7a5edc8a9d8d1ff67fae83ab36""",0.463015,STANDARD,154,96,154_96,ount_clear.tif,2014
4,wofs_ls_summary_annual/1-0-0/x154/y096/2014--P...,2021-09-08 02:35:40+00:00,"""84adbc9a2c8ede3d1addd8b87a7d5ff5""",0.392237,STANDARD,154,96,154_96,_count_wet.tif,2014


In [28]:
s3res['YEAR'].value_counts()

2013    13029
2014    13002
2015    12981
2017    12966
2021    12957
2019    12948
2018    12942
2016    12942
2020    12933
2006    12867
2005    12807
2012    12807
2000    12798
2001    12795
2007    12777
2002    12771
2009    12771
2011    12753
2003    12732
2010    12726
2008    12693
1999    12660
2004    12648
1998    11673
1984    11643
1995    11469
1987    11460
1994    11313
1986    11301
1990    10341
1985     9600
1989     9585
1991     8889
1996     8829
1997     8643
1993     7896
1992     7821
1988     6492
Name: YEAR, dtype: int64

In [18]:
s3res.loc[s3res['TYPE'] == "_frequency.tif"].groupby(['TYPE','YEAR']).sum()

,,Size,COL,ROW
TYPE,YEAR,,,
_frequency.tif,1984,47343.836777,882439,365732


# Download all data for specific year and type

In [19]:
# Download all time summaries
sel_type = "_frequency.tif"
to_download = s3res.loc[(s3res['TYPE'] == sel_type)]
out_folder = f"/home/wb411133/temp/WOFS_Frequency_AllTime"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

cnt = 0
for idx, row in to_download.iterrows():
    out_file = os.path.join(out_folder, os.path.basename(row['Key']))
    if not os.path.exists(out_file):
        s3client.download_file(bucket, row['Key'], out_file)
    if cnt % 1000 == 0:
        tPrint(f'{year}: downloading {cnt} of {to_download.shape[0]}')
    cnt += 1

16:32:54	2020: downloading 0 of 4394
16:58:55	2020: downloading 1000 of 4394
17:27:46	2020: downloading 2000 of 4394
17:57:43	2020: downloading 3000 of 4394
18:31:03	2020: downloading 4000 of 4394


In [29]:
# Download annual summaries

for year in range(1984, 2001):
    sel_type = "_frequency.tif"
    to_download = s3res.loc[(s3res['TYPE'] == sel_type) & (s3res['YEAR'] == str(year))]

    out_folder = f"/home/wb411133/temp/WOFS_Frequency_{year}"
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
        
    cnt = 0
    for idx, row in to_download.iterrows():
        out_file = os.path.join(out_folder, os.path.basename(row['Key']))
        if not os.path.exists(out_file):
            s3client.download_file(bucket, row['Key'], out_file)
        if cnt % 1000 == 0:
            tPrint(f'{year}: downloading {cnt} of {to_download.shape[0]}')
        cnt += 1
        
            

09:12:17	1984: downloading 0 of 3881
09:25:31	1984: downloading 1000 of 3881
09:37:22	1984: downloading 2000 of 3881
09:48:57	1984: downloading 3000 of 3881
09:58:42	1985: downloading 0 of 3200
10:10:49	1985: downloading 1000 of 3200
10:23:38	1985: downloading 2000 of 3200
10:35:26	1985: downloading 3000 of 3200
10:37:52	1986: downloading 0 of 3767
10:48:00	1986: downloading 1000 of 3767
11:02:22	1986: downloading 2000 of 3767
11:12:53	1986: downloading 3000 of 3767
11:22:01	1987: downloading 0 of 3820
11:34:00	1987: downloading 1000 of 3820
11:46:20	1987: downloading 2000 of 3820
11:57:51	1987: downloading 3000 of 3820
12:08:59	1988: downloading 0 of 2164
12:21:15	1988: downloading 1000 of 2164
12:36:32	1988: downloading 2000 of 2164
12:38:40	1989: downloading 0 of 3195
12:50:54	1989: downloading 1000 of 3195
13:04:27	1989: downloading 2000 of 3195
13:18:54	1989: downloading 3000 of 3195
13:22:19	1990: downloading 0 of 3447
13:33:12	1990: downloading 1000 of 3447
13:47:15	1990: downlo

In [ ]:
to_download.iloc[0,0]

# Download  data for specific area

In [ ]:
# Limit s3 results to selected area
cur_s3 = s3res.loc[s3res['COL_ROW'].isin(sel_tiles['COL_ROW'])]
cur_s3 = cur_s3.loc[(cur_s3['YEAR'] == str(year)) & (cur_s3['TYPE'] == product)]

In [ ]:
# Download the results
loop = 0
for idx, row in cur_s3.iterrows():
    out_file = os.path.join(out_folder, os.path.basename(row['Key']))
    s3client.download_file(bucket, row['Key'], out_file)
    loop += 1
    print(f'{loop} of {cur_s3.shape}')